In [1]:
#| default_exp notebook_context

## Jupyter Kernels detailed technical implementation

https://chatgpt.com/share/692bea08-4510-8004-b9ab-c02feeb97c08

## Jupyterlab extension development tutorial

https://jupyterlab.readthedocs.io/en/latest/extension/extension_tutorial.html

## Build this extension

### Edit wordslab_notebooks_lib/__init__.py

Add this function definition:

```python
def _jupyter_labextension_paths():
    return [{
        "src": "labextension",
        "dest": "wordslab-notebooks-extension"
    }]
```

This is what Jupyterlab uses in development to find the Jupyterlab extension in a module.

### Edit settings.ini

Add this classifier for Pypi:

> Framework :: Jupyter :: JupyterLab :: Extensions :: Prebuilt

This is what Jupyterlab uses in production to find python packages with Jupyterlab extensions.

### Build everything

Open a Terminal

```bash

cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib

source ../../jupyterlab/.venv/bin/activate

# Install dependencies
jlpm clean
jlpm install

# Build TypeScript extension
jlpm build

source .venv/bin/activate

# Export notebooks to Python modules
nbdev_export

# Prepare for release
nbdev_prepare
```

### Test locally

```bash
source ../../jupyterlab/.venv/bin/activate

# Install in development mode
uv pip install -e .

# Register the extension with JupyterLab during development
jupyter labextension develop . --overwrite

# Verify extension is found
jupyter labextension list

# Start JupyterLab
jupyter lab
```

### When ready to publish:

```bash
source ../../jupyterlab/.venv/bin/activate

# Make sure extension is built
jlpm build

source .venv/bin/activate

# Publish to PyPI
nbdev_pypi
```

In [1]:
#| export
import asyncio
from ipykernel.comm import Comm
from IPython import get_ipython

class NotebookContextManager:
    def __init__(self):
        self.comm_target = 'notebook_context_comm'
        self._comm = None
        self._registered = False
    
    def _ensure_registered(self):
        if not self._registered:
            self._comm = Comm(target_name=self.comm_target)
            self._registered = True
    
    async def get_notebook_context(self):
        self._ensure_registered()
        
        future = asyncio.Future()
        
        def handle_response(msg):
            if not future.done():
                future.set_result(msg['content']['data']['cells'])
        
        self._comm.on_msg(handle_response)
        self._comm.send({'action': 'request_cells'})
        
        try:
            result = await asyncio.wait_for(future, timeout=5.0)
            return result
        except asyncio.TimeoutError:
            raise TimeoutError("Failed to receive notebook context from frontend")
    
    def get_notebook_context_sync(self):
        try:
            import nest_asyncio
            nest_asyncio.apply()
        except ImportError:
            raise ImportError("nest_asyncio is required. Install with: pip install nest_asyncio")
        
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self.get_notebook_context())
    
    def format_context_for_llm(self, cells, include_outputs=True):
        context_parts = []
        
        for i, cell in enumerate(cells):
            if cell['type'] == 'markdown':
                context_parts.append(f"## Markdown Cell {i+1}")
                context_parts.append(cell['source'])
            elif cell['type'] == 'code':
                context_parts.append(f"## Code Cell {i+1}")
                context_parts.append(f"```python\n{cell['source']}\n```")
                
                if include_outputs and cell['outputs']:
                    context_parts.append("### Output:")
                    for output in cell['outputs']:
                        if output.get('text'):
                            context_parts.append(output['text'])
                        elif output.get('data'):
                            data = output['data']
                            if 'text/plain' in data:
                                context_parts.append(data['text/plain'])
            
            context_parts.append("")
        
        return "\n".join(context_parts)

_manager = NotebookContextManager()

async def get_notebook_context():
    return await _manager.get_notebook_context()

def get_notebook_context_sync():
    return _manager.get_notebook_context_sync()

def format_for_llm(include_outputs=True):
    cells = _manager.get_notebook_context_sync()
    return _manager.format_context_for_llm(cells, include_outputs)


In [12]:
#get_notebook_context_sync()

In [23]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from ipykernel.comm import Comm

def get_notebook_data(timeout=10):
    result = {}
    
    def on_msg(msg):
        result['data'] = msg['content']['data']
    
    comm = Comm(target_name='wordslab_notebook_comm')
    comm.on_msg(on_msg)
    comm.open()
    comm.send({'request': 'get_notebook_data'})
    
    loop = asyncio.get_event_loop()
    elapsed = 0
    interval = 0.05
    
    while 'data' not in result and elapsed < timeout:
        loop.run_until_complete(asyncio.sleep(interval))
        elapsed += interval
    
    if 'data' not in result:
        raise TimeoutError("No response from frontend extension")
    
    return result['data']

In [24]:
get_notebook_data()

/tmp/ipykernel_11668/4097989723.py:13: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  comm = Comm(target_name='wordslab_notebook_comm')


{'notebook': {'metadata': {'kernelspec': {'display_name': 'wordslab-notebooks-lib',
    'language': 'python',
    'name': 'wordslab-notebooks-lib'},
   'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
    'file_extension': '.py',
    'mimetype': 'text/x-python',
    'name': 'python',
    'nbconvert_exporter': 'python',
    'pygments_lexer': 'ipython3',
    'version': '3.12.12'}},
  'nbformat_minor': 5,
  'nbformat': 4,
  'cells': [{'id': '845d409b-9d48-4b97-9af6-63143b61e9fa',
    'cell_type': 'code',
    'source': '#| default_exp notebook_context',
    'metadata': {'execution': {'iopub.execute_input': '2025-11-28T22:44:11.599735Z',
      'iopub.status.busy': '2025-11-28T22:44:11.599597Z',
      'iopub.status.idle': '2025-11-28T22:44:11.602236Z',
      'shell.execute_reply': '2025-11-28T22:44:11.601449Z',
      'shell.execute_reply.started': '2025-11-28T22:44:11.599725Z'},
     'trusted': True},
    'outputs': [],
    'execution_count': 1},
   {'id': '05977d50-8c

In [25]:
get_notebook_data()

/tmp/ipykernel_11668/4097989723.py:13: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  comm = Comm(target_name='wordslab_notebook_comm')


{'notebook': {'metadata': {'kernelspec': {'display_name': 'wordslab-notebooks-lib',
    'language': 'python',
    'name': 'wordslab-notebooks-lib'},
   'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
    'file_extension': '.py',
    'mimetype': 'text/x-python',
    'name': 'python',
    'nbconvert_exporter': 'python',
    'pygments_lexer': 'ipython3',
    'version': '3.12.12'}},
  'nbformat_minor': 5,
  'nbformat': 4,
  'cells': [{'id': '845d409b-9d48-4b97-9af6-63143b61e9fa',
    'cell_type': 'code',
    'source': '#| default_exp notebook_context',
    'metadata': {'execution': {'iopub.execute_input': '2025-11-28T22:44:11.599735Z',
      'iopub.status.busy': '2025-11-28T22:44:11.599597Z',
      'iopub.status.idle': '2025-11-28T22:44:11.602236Z',
      'shell.execute_reply': '2025-11-28T22:44:11.601449Z',
      'shell.execute_reply.started': '2025-11-28T22:44:11.599725Z'},
     'trusted': True},
    'outputs': [],
    'execution_count': 1},
   {'id': '05977d50-8c